In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os, requests, tarfile, sys
from pathlib import Path

# === config ===
BASE_DIR = Path("/content/drive/MyDrive/nasa_exoplanet/Microlensing")
DATASET_DIR = BASE_DIR / "roman_2018"
DATASET_DIR.mkdir(parents=True, exist_ok=True)

FILES = {
    # Lightcurves tarball (unzips to ./lc/ with text files per event)
    "lc.tar.gz": "https://github.com/microlensing-data-challenge/data-challenge-1/raw/master/lc.tar.gz",
    # Event + ephemerides (plain text)
    "event_info.txt": "https://raw.githubusercontent.com/microlensing-data-challenge/data-challenge-1/master/event_info.txt",
    "wfirst_ephemeris_W149.txt": "https://raw.githubusercontent.com/microlensing-data-challenge/data-challenge-1/master/wfirst_ephemeris_W149.txt",
    "wfirst_ephemeris_Z087.txt": "https://raw.githubusercontent.com/microlensing-data-challenge/data-challenge-1/master/wfirst_ephemeris_Z087.txt",
}

def download(url: str, dest: Path, chunk=1<<15):
    dest.parent.mkdir(parents=True, exist_ok=True)
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        total = int(r.headers.get("content-length", 0))
        done = 0
        with open(dest, "wb") as f:
            for part in r.iter_content(chunk_size=chunk):
                if part:
                    f.write(part)
                    done += len(part)
                    if total:
                        pct = (done/total)*100
                        sys.stdout.write(f"\r↓ {dest.name}: {pct:5.1f}%")
                        sys.stdout.flush()
    size_mb = dest.stat().st_size / 1e6
    print(f"\r✓ {dest.name} saved ({size_mb:.1f} MB)")

# 1) download all files if missing
for name, url in FILES.items():
    path = DATASET_DIR / name
    if not path.exists():
        download(url, path)
    else:
        print(f"• {name} already exists, skipping")

# 2) extract lightcurves safely into DATASET_DIR/lc/
tar_path = DATASET_DIR / "lc.tar.gz"
extract_root = DATASET_DIR

def safe_extract(tar: tarfile.TarFile, path: Path = Path(".")):
    path = path.resolve()
    for member in tar.getmembers():
        member_path = (path / member.name).resolve()
        if not str(member_path).startswith(str(path)):
            raise Exception("Blocked path traversal in tar file")
    tar.extractall(path)

if tar_path.exists():
    with tarfile.open(tar_path, "r:gz") as tar:
        safe_extract(tar, extract_root)
    print("✓ Extracted lightcurves to:", extract_root / "lc")
else:
    print("WARN: lc.tar.gz missing?")

# 3) quick sanity: count files per filter
from glob import glob
lc_dir = DATASET_DIR / "lc"
w149 = glob(str(lc_dir / "*_W149.txt"))
z087 = glob(str(lc_dir / "*_Z087.txt"))
print(f"Found {len(w149)} W149 files and {len(z087)} Z087 files at {lc_dir}")


✓ lc.tar.gz saved (101.8 MB)
✓ event_info.txt saved (0.0 MB)
✓ wfirst_ephemeris_W149.txt saved (2.8 MB)
✓ wfirst_ephemeris_Z087.txt saved (0.1 MB)


/tmp/ipython-input-1729497127.py:54: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path)


✓ Extracted lightcurves to: /content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/lc
Found 293 W149 files and 293 Z087 files at /content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/lc


In [ ]:
import os

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

folder_to_check = "/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018"
size_in_bytes = get_folder_size(folder_to_check)

# Convert bytes to megabytes for better readability
size_in_mb = size_in_bytes / (1024 * 1024)

print(f"The size of the folder '{folder_to_check}' is: {size_in_mb:.2f} MB")

The size of the folder '/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018' is: 474.58 MB


In [ ]:
# ============================================
#  Roman 2018 Microlensing -> Train-Ready Tensors
#  - Pairs W149 & Z087 per event
#  - Cleans, normalizes, resamples to fixed length
#  - Exports: numpy arrays + manifest.csv + JSON meta
#  Path you gave: "/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018"
# ============================================

# --- Colab Drive mount (safe to skip locally) ---
try:
    from google.colab import drive
    drive.mount("/content/drive")
except Exception:
    pass

import os, re, sys, math, json, gzip, shutil, textwrap, warnings
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import Optional, Dict, List, Tuple

import numpy as np
import pandas as pd

# Optional: tqdm for pretty progress (falls back to no-op)
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **k): return x

# -----------------------------
#           CONFIG
# -----------------------------
BASE_DIR = Path("/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018")
LC_DIR   = BASE_DIR / "lc"   # where the extracted txt files live
OUT_DIR  = BASE_DIR / "processed"

# target sequence length (per filter). Keep 512 for nice resolution; drop to 256 if you need speed.
T = 512

# channel layout: 2 channels if both filters present. If one filter missing, we still emit 2 with NaNs -> 0.
CHANNELS = ["W149", "Z087"]

# outlier cleaning (magnitude domain) – MAD-based clip (robust)
MAD_CLIP_SIGMA = 7.0

# minimum number of points per filter to accept (after cleaning)
MIN_POINTS = 30

# random seed for any deterministic ordering
RNG_SEED = 1337
np.random.seed(RNG_SEED)

# -----------------------------
#       UTILITIES
# -----------------------------
EVENT_RE = re.compile(r"^(ulwdc1_\d+)_([A-Z0-9]+)\.txt$", re.IGNORECASE)

def safe_mkdir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def read_lightcurve_txt(path: Path) -> pd.DataFrame:
    """
    Roman 2018 lightcurves: 3 cols -> BJD Magnitude Error (whitespace-delimited).
    Returns sorted DataFrame with columns ['BJD','mag','err'].
    """
    df = pd.read_csv(path, delim_whitespace=True, comment="#", header=None,
                     names=["BJD","mag","err"], dtype=float)
    df = df.dropna().sort_values("BJD").reset_index(drop=True)
    return df

def mag_to_flux(mag: np.ndarray) -> np.ndarray:
    # astronomical convention (relative): F ~ 10^(-0.4 * mag)
    return np.power(10.0, -0.4 * mag, dtype=np.float64)

def robust_mad_clip_mag(df: pd.DataFrame, sigma: float = MAD_CLIP_SIGMA) -> pd.DataFrame:
    """
    Clip outliers in magnitudes using Median Absolute Deviation.
    Returns filtered DataFrame.
    """
    mags = df["mag"].values
    med  = np.median(mags)
    mad  = np.median(np.abs(mags - med)) + 1e-12
    z = 0.6745 * (mags - med) / mad
    keep = np.abs(z) <= sigma
    return df.loc[keep].reset_index(drop=True)

def estimate_peak_time(df: pd.DataFrame) -> float:
    """
    Peak microlensing -> minimum magnitude (brightest). If working in flux, it's max.
    """
    idx = int(np.argmin(df["mag"].values))
    return float(df.loc[idx, "BJD"])

def resample_to_grid(t: np.ndarray, y: np.ndarray, T: int, tmin: float, tmax: float) -> np.ndarray:
    """
    Resample irregular (t,y) onto uniform grid in [tmin, tmax] using linear interpolation.
    """
    if tmax <= tmin:
        tmax = tmin + 1.0
    grid = np.linspace(tmin, tmax, T, dtype=np.float64)
    # numpy.interp extrapolates with edges -> that's okay; we'll clip to finite after standardization
    out = np.interp(grid, t, y, left=np.nan, right=np.nan)
    # Fix leading/trailing NaNs by nearest non-NaN fill
    if np.any(np.isnan(out)):
        # forward fill then back fill
        nans = np.isnan(out)
        if np.all(nans):
            # hopeless: return zeros (caller may handle)
            out = np.zeros_like(out, dtype=np.float64)
        else:
            # forward fill
            for i in range(1, len(out)):
                if np.isnan(out[i]) and not np.isnan(out[i-1]):
                    out[i] = out[i-1]
            # back fill
            for i in range(len(out)-2, -1, -1):
                if np.isnan(out[i]) and not np.isnan(out[i+1]):
                    out[i] = out[i+1]
            # any remaining NaN -> zeros
            out = np.nan_to_num(out, nan=0.0)
    return out

def standardize(x: np.ndarray) -> np.ndarray:
    mu = np.mean(x)
    sd = np.std(x)
    return (x - mu) / (sd + 1e-8)

@dataclass
class EventRecord:
    event_id: str
    has_W149: bool
    has_Z087: bool
    n_W149: int
    n_Z087: int
    span_W149: float
    span_Z087: float
    t0_W149: Optional[float]
    t0_Z087: Optional[float]
    label: int  # -1 = unknown, 0/1 reserved for non-planet/planet if you add labels later
    notes: str = ""

# -----------------------------
#   DISCOVER + PAIR FILES
# -----------------------------
assert LC_DIR.exists(), f"Missing lightcurve dir: {LC_DIR}"

pairs: Dict[str, Dict[str, Path]] = {}
for p in LC_DIR.glob("*.txt"):
    m = EVENT_RE.match(p.name)
    if not m:
        continue
    key, fil = m.group(1), m.group(2).upper()
    if fil not in CHANNELS:
        continue
    pairs.setdefault(key, {})[fil] = p

event_ids = sorted(pairs.keys())
print(f"Discovered {len(event_ids)} events (keys), expected ~293. Example:", event_ids[:5])

# -----------------------------
#   OPTIONAL: READ EVENT INFO
#   (extinction etc., no labels)
# -----------------------------
event_info_path = BASE_DIR / "event_info.txt"
event_info: Optional[pd.DataFrame] = None
if event_info_path.exists():
    # event_info.txt has columns listed in README (no header row in file),
    # so we parse by whitespace and name columns explicitly:
    colnames = ["Event_name","Event_number","RA_deg","Dec_deg",
                "Distance","A_W149","sigma_A_W149","A_Z087","sigma_A_Z087"]
    try:
        event_info = pd.read_csv(event_info_path, delim_whitespace=True,
                                 names=colnames, comment="#", header=None)
        # Normalize event key to match 'ulwdc1_xxx' pattern if present
        # Heuristic: try to map Event_name or number to our keys – we leave it for metadata only.
    except Exception as e:
        print("WARN: could not parse event_info.txt ->", e)
else:
    print("NOTE: event_info.txt not found (that’s fine).")

# -----------------------------
#   PROCESS ALL EVENTS
# -----------------------------
safe_mkdir(OUT_DIR)

X_list_2ch: List[np.ndarray] = []  # (T, 2)
X_list_1ch: List[np.ndarray] = []  # (T, 1) fallback when only one filter is usable
Y_list: List[int] = []             # labels (unknown = -1)
REC_list: List[EventRecord] = []   # manifest rows

def clean_and_resample(path: Path, t0_hint: Optional[float]) -> Tuple[np.ndarray, float, float, Optional[float]]:
    """
    - load df
    - mad clip in mag
    - convert to flux
    - define window [tmin, tmax] around the shared times (handled outside)
    - resampling is done outside (we return raw t,flux for both filters)
    Here we just return cleaned arrays.
    """
    df = read_lightcurve_txt(path)
    if len(df) == 0:
        return np.array([]), np.nan, np.nan, None
    df = robust_mad_clip_mag(df, sigma=MAD_CLIP_SIGMA)
    if len(df) < MIN_POINTS:
        return np.array([]), np.nan, np.nan, None
    t = df["BJD"].values.astype(np.float64)
    f = mag_to_flux(df["mag"].values.astype(np.float64))
    # rough t0 -> brightest point (min mag == max flux)
    t0 = t[np.argmax(f)]
    return np.stack([t, f], axis=1), float(t.min()), float(t.max()), t0

for eid in tqdm(event_ids, desc="Processing events"):
    ch_paths = pairs[eid]
    # prepare containers
    per_filter = {}
    tspans = {}
    t0s = {}

    # 1) clean each present filter
    for fil in CHANNELS:
        if fil in ch_paths:
            arr, tmin, tmax, t0 = clean_and_resample(ch_paths[fil], t0_hint=None)
            per_filter[fil] = arr  # shape (N, 2) with columns [t, flux]
            tspans[fil] = (tmin, tmax)
            t0s[fil] = t0
        else:
            per_filter[fil] = None

    has_W = per_filter["W149"] is not None and len(per_filter["W149"]) >= MIN_POINTS
    has_Z = per_filter["Z087"] is not None and len(per_filter["Z087"]) >= MIN_POINTS

    nW = int(per_filter["W149"].shape[0]) if has_W else 0
    nZ = int(per_filter["Z087"].shape[0]) if has_Z else 0
    spanW = (tspans["W149"][1] - tspans["W149"][0]) if has_W else 0.0
    spanZ = (tspans["Z087"][1] - tspans["Z087"][0]) if has_Z else 0.0

    # 2) define a common time window for resampling
    if has_W and has_Z:
        # overlap window (safer for true 2-channel sync)
        tmin = max(tspans["W149"][0], tspans["Z087"][0])
        tmax = min(tspans["W149"][1], tspans["Z087"][1])
    elif has_W:
        tmin, tmax = tspans["W149"]
    elif has_Z:
        tmin, tmax = tspans["Z087"]
    else:
        # nothing usable -> skip
        REC_list.append(EventRecord(
            event_id=eid, has_W149=False, has_Z087=False, n_W149=nW, n_Z087=nZ,
            span_W149=spanW, span_Z087=spanZ, t0_W149=None, t0_Z087=None,
            label=-1, notes="no_usable_points"
        ))
        continue

    if not np.isfinite(tmin) or not np.isfinite(tmax) or (tmax - tmin) <= 0:
        REC_list.append(EventRecord(
            event_id=eid, has_W149=has_W, has_Z087=has_Z, n_W149=nW, n_Z087=nZ,
            span_W149=spanW, span_Z087=spanZ,
            t0_W149=(float(t0s["W149"]) if has_W else None),
            t0_Z087=(float(t0s["Z087"]) if has_Z else None),
            label=-1, notes="bad_time_window"
        ))
        continue

    # 3) resample per filter to uniform grid then z-score per channel
    def resample_channel(fil):
        arr = per_filter[fil]
        if arr is None or len(arr) < MIN_POINTS:
            return None
        t = arr[:, 0]
        y = arr[:, 1]
        y_grid = resample_to_grid(t, y, T=T, tmin=tmin, tmax=tmax)
        y_grid = standardize(y_grid)
        return y_grid.astype(np.float32)

    yW = resample_channel("W149")
    yZ = resample_channel("Z087")

    if (yW is not None) and (yZ is not None):
        X = np.stack([yW, yZ], axis=1)  # shape (T, 2)
        X_list_2ch.append(X)
        Y_list.append(-1)  # unknown labels (you can fill later)
    else:
        # at least one usable channel -> create (T,1)
        y_one = yW if yW is not None else yZ
        if y_one is None:
            REC_list.append(EventRecord(
                event_id=eid, has_W149=has_W, has_Z087=has_Z, n_W149=nW, n_Z087=nZ,
                span_W149=spanW, span_Z087=spanZ,
                t0_W149=(float(t0s["W149"]) if has_W else None),
                t0_Z087=(float(t0s["Z087"]) if has_Z else None),
                label=-1, notes="channels_too_short"
            ))
            continue
        X_list_1ch.append(y_one[:, None])  # (T,1)
        Y_list.append(-1)

    REC_list.append(EventRecord(
        event_id=eid, has_W149=(yW is not None), has_Z087=(yZ is not None),
        n_W149=nW, n_Z087=nZ, span_W149=spanW, span_Z087=spanZ,
        t0_W149=(float(t0s["W149"]) if has_W else None),
        t0_Z087=(float(t0s["Z087"]) if has_Z else None),
        label=-1, notes=""
    ))

# -----------------------------
#   STACK + SAVE ARTIFACTS
# -----------------------------
safe_mkdir(OUT_DIR)

X2 = np.stack(X_list_2ch, axis=0) if len(X_list_2ch) else np.empty((0, T, 2), dtype=np.float32)
X1 = np.stack(X_list_1ch, axis=0) if len(X_list_1ch) else np.empty((0, T, 1), dtype=np.float32)
Y  = np.array(Y_list, dtype=np.int8) if len(Y_list) else np.empty((0,), dtype=np.int8)

np.save(OUT_DIR / "X_2ch.npy", X2)   # events with both W149+Z087
np.save(OUT_DIR / "X_1ch.npy", X1)   # events with only one filter usable
np.save(OUT_DIR / "y.npy", Y)        # -1 = unknown (fill later if you add labels)

# manifest
manifest_rows = [asdict(r) for r in REC_list]
manifest_df = pd.DataFrame(manifest_rows)
manifest_df.to_csv(OUT_DIR / "manifest.csv", index=False)

# meta
meta = {
    "description": "Roman 2018 microlensing lightcurves processed for ML (no labels).",
    "source": "https://github.com/microlensing-data-challenge/data-challenge-1",
    "sequence_length_T": T,
    "channels": CHANNELS,
    "standardization": "per-channel z-score after flux conversion and uniform resampling",
    "time_window": "per-event, overlap of filters if both present; else full filter span",
    "label_semantics": {"-1": "unknown", "0": "non-planet", "1": "planet"},
    "created_with_seed": RNG_SEED,
}
with open(OUT_DIR / "meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print("\n=== SUMMARY ===")
print(f"2-channel samples: {X2.shape} -> saved to {OUT_DIR/'X_2ch.npy'}")
print(f"1-channel samples: {X1.shape} -> saved to {OUT_DIR/'X_1ch.npy'}")
print(f"Labels (y):       {Y.shape}  -> saved to {OUT_DIR/'y.npy'}")
print(f"Manifest rows:    {len(REC_list)} -> {OUT_DIR/'manifest.csv'}")
print(f"Meta:             {OUT_DIR/'meta.json'}")

# -----------------------------
#  OPTIONAL: quick sanity plots
#  (uncomment if you want quick looks; no training)
# -----------------------------
# import matplotlib.pyplot as plt
# if X2.shape[0] > 0:
#     idx = 0
#     plt.figure(figsize=(10,3))
#     plt.plot(X2[idx,:,0], label="W149")
#     plt.plot(X2[idx,:,1], label="Z087", alpha=0.7)
#     plt.legend(); plt.title("Example 2-channel event (standardized)")
#     plt.show()
#
# if X1.shape[0] > 0:
#     idx = 0
#     plt.figure(figsize=(10,3))
#     plt.plot(X1[idx,:,0], label="Single filter")
#     plt.legend(); plt.title("Example 1-channel event (standardized)")
#     plt.show()

# -----------------------------
#  (Bonus) HOW TO ADD LABELS LATER — two easy options (not executed):
# -----------------------------
# 1) Inject synthetic "planet bumps" into a fraction of normal curves, mark them as 1.
# 2) If you acquire a CSV mapping event_id -> {0,1}, just:
#    labels = pd.read_csv("labels.csv")
#    y_map = dict(zip(labels["event_id"], labels["label"]))
#    manifest = pd.read_csv(OUT_DIR / "manifest.csv")
#    new_y = [y_map.get(eid, -1) for eid in manifest["event_id"]]
#    np.save(OUT_DIR / "y.npy", np.array(new_y, dtype=np.int8))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Discovered 293 events (keys), expected ~293. Example: ['ulwdc1_001', 'ulwdc1_002', 'ulwdc1_003', 'ulwdc1_004', 'ulwdc1_005']


/tmp/ipython-input-268114996.py:172: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  event_info = pd.read_csv(event_info_path, delim_whitespace=True,


Processing events:   0%|          | 0/293 [00:00<?, ?it/s]

/tmp/ipython-input-268114996.py:66: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, comment="#", header=None,
/tmp/ipython-input-268114996.py:66: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, comment="#", header=None,
/tmp/ipython-input-268114996.py:66: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, comment="#", header=None,
/tmp/ipython-input-268114996.py:66: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, comment="#", heade


=== SUMMARY ===
2-channel samples: (293, 512, 2) -> saved to /content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/X_2ch.npy
1-channel samples: (0, 512, 1) -> saved to /content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/X_1ch.npy
Labels (y):       (293,)  -> saved to /content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/y.npy
Manifest rows:    293 -> /content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/manifest.csv
Meta:             /content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/meta.json


/tmp/ipython-input-268114996.py:66: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, comment="#", header=None,
/tmp/ipython-input-268114996.py:66: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, comment="#", header=None,
/tmp/ipython-input-268114996.py:66: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, comment="#", header=None,


In [ ]:
# =============================
# Roman Microlensing: Conv1D → GRU trainer (90/10 split)
# - Uses two-channel sequences (W149, Z087), length 512
# - If y has no 0/1 labels, it injects synthetic planet bumps
# - Saves model and prints test metrics
# =============================

from pathlib import Path
import numpy as np, math
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ---- Config
PROC_DIR = Path("/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed")
OUT_DIR  = PROC_DIR / "models"
OUT_DIR.mkdir(exist_ok=True, parents=True)
SEED = 1337
BATCH = 64
EPOCHS = 15
LR = 3e-4
torch.manual_seed(SEED); np.random.seed(SEED)

# ---- Load tensors
X2 = np.load(PROC_DIR/"X_2ch.npy")        # (N, 512, 2) float32
y  = np.load(PROC_DIR/"y.npy")            # (-1/0/1)    int8 or int32
N  = len(X2)
print("Loaded:", X2.shape, "labels:", y.shape)

# ---- Build labels: use real if present, else synth
has_real = np.isin(y, [0,1]).sum() >= 10
if has_real:
    print("Using REAL labels found in y.npy")
    mask = np.isin(y, [0,1])
    X_use = X2[mask].copy()
    y_use = y[mask].astype(np.float32)
else:
    print("No usable labels detected → generating SYNTHETIC planet anomalies")
    rng = np.random.default_rng(SEED)
    X_use = X2.copy()
    y_use = np.zeros(len(X_use), dtype=np.float32)

    # choose ~35% positives
    pos_idx = rng.choice(len(X_use), size=max(int(0.35*len(X_use)), 1), replace=False)
    y_use[pos_idx] = 1.0

    # inject a Gaussian "planet bump" into positives (flux domain; your X is normalized)
    T = X_use.shape[1]
    t = np.arange(T)
    for i in pos_idx:
        t0 = rng.integers(low=64, high=T-64)     # keep away from edges
        width = rng.integers(3, 12)              # how wide the anomaly is
        amp = rng.uniform(1.0, 2.5)              # ~1–2.5 sigma in normalized units
        bump = np.exp(-0.5*((t - t0)/width)**2) * amp
        ch_scale = rng.uniform(0.85, 1.15)       # slight channel mismatch
        X_use[i, :, 0] += bump
        X_use[i, :, 1] += bump * ch_scale

print(f"Data ready: {X_use.shape}, positives={int(y_use.sum())}, negatives={len(y_use)-int(y_use.sum())}")

# ---- Stratified 90/10 split
def stratified_split(y, train_ratio=0.9, seed=SEED):
    rng = np.random.default_rng(seed)
    idx_pos = np.where(y == 1.0)[0]
    idx_neg = np.where(y == 0.0)[0]
    rng.shuffle(idx_pos); rng.shuffle(idx_neg)
    ntr_pos = int(train_ratio*len(idx_pos))
    ntr_neg = int(train_ratio*len(idx_neg))
    tr = np.concatenate([idx_pos[:ntr_pos], idx_neg[:ntr_neg]])
    te = np.concatenate([idx_pos[ntr_pos:], idx_neg[ntr_neg:]])
    rng.shuffle(tr); rng.shuffle(te)
    return tr, te

tr_idx, te_idx = stratified_split(y_use, 0.9, SEED)
print(f"Split → train={len(tr_idx)} | test={len(te_idx)}")

# ---- PyTorch dataset
class TwoChSeqDS(Dataset):
    def __init__(self, X, y, idx):
        self.X = X[idx]
        self.y = y[idx]
    def __len__(self): return len(self.y)
    def __getitem__(self, i):
        # model expects (C,T)
        x = self.X[i].transpose(1,0)  # (2,512)
        return torch.from_numpy(x).float(), torch.tensor(self.y[i], dtype=torch.float32)

train_loader = DataLoader(TwoChSeqDS(X_use, y_use, tr_idx), batch_size=BATCH, shuffle=True,  num_workers=2, pin_memory=True)
test_loader  = DataLoader(TwoChSeqDS(X_use, y_use, te_idx), batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)

# ---- Model: Conv1D → GRU → Dense
class Conv1DGRU(nn.Module):
    def __init__(self, in_ch=2, hid=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_ch, 32, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),                 # 512 -> 256
            nn.Conv1d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
        )
        self.gru = nn.GRU(input_size=64, hidden_size=64, batch_first=True, bidirectional=True)
        self.head = nn.Sequential(
            nn.Linear(64*2, 64), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
    def forward(self, x):                    # x: (B, 2, 512)
        z = self.conv(x)                     # (B, 64, 256)
        z = z.transpose(1, 2)                # (B, 256, 64)
        z, _ = self.gru(z)                   # (B, 256, 128)
        z = z.mean(dim=1)                    # (B, 128)
        return self.head(z).squeeze(1)       # (B,)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Conv1DGRU().to(device)
opt   = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

# class imbalance handling
pos = float(y_use[tr_idx].sum())
neg = float(len(tr_idx) - pos)
pos_weight = torch.tensor([neg / max(pos, 1.0)], device=device)
criterion  = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
print(f"pos_weight = {pos_weight.item():.2f}")

def run_epoch(loader, train=True):
    model.train(train)
    total, n = 0.0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        if train: opt.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        if train:
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
        total += loss.item() * xb.size(0); n += xb.size(0)
    return total / max(n,1)

best, best_state = 1e9, None
for ep in range(1, EPOCHS+1):
    tr_loss = run_epoch(train_loader, True)
    te_loss = run_epoch(test_loader,  False)
    print(f"epoch {ep:02d} | train {tr_loss:.4f} | test {te_loss:.4f}")
    if te_loss < best:
        best, best_state = te_loss, {k: v.detach().cpu().clone() for k,v in model.state_dict().items()}

if best_state: model.load_state_dict(best_state)

# ---- Metrics on test
# try sklearn for PR-AUC/F1; fallback to manual if not available
try:
    from sklearn.metrics import average_precision_score, confusion_matrix, classification_report
    SK = True
except Exception:
    SK = False

model.eval()
all_probs, all_true = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        probs = torch.sigmoid(model(xb.to(device))).cpu().numpy()
        all_probs.append(probs); all_true.append(yb.numpy())
all_probs = np.concatenate(all_probs)
all_true = np.concatenate(all_true).astype(int)
preds = (all_probs >= 0.5).astype(int)

print(f"Test positives: {all_true.sum()} / {len(all_true)}")
if SK:
    ap = average_precision_score(all_true, all_probs)
    print(f"PR-AUC (Average Precision): {ap:.3f}")
    print("Confusion matrix:\n", confusion_matrix(all_true, preds))
    print(classification_report(all_true, preds, digits=3))
else:
    tp = ((preds==1)&(all_true==1)).sum()
    fp = ((preds==1)&(all_true==0)).sum()
    fn = ((preds==0)&(all_true==1)).sum()
    prec = tp/(tp+fp+1e-9); rec = tp/(tp+fn+1e-9)
    f1 = 2*prec*rec/(prec+rec+1e-9)
    print(f"Precision {prec:.3f} | Recall {rec:.3f} | F1 {f1:.3f}")

# ---- Save model
SAVE_PATH = OUT_DIR / "conv1d_gru.pt"
torch.save({"state_dict": model.state_dict(), "config": {"in_ch":2, "hid":64}}, SAVE_PATH)
print("Saved model ->", SAVE_PATH)

# ---- Inference helper
def score_events(X):
    """Return planet probabilities for a batch of two-channel curves shaped (N, 512, 2)."""
    dl = DataLoader(TwoChSeqDS(X, np.zeros(len(X)), np.arange(len(X))), batch_size=128, shuffle=False)
    probs = []
    model.eval()
    with torch.no_grad():
        for xb,_ in dl:
            p = torch.sigmoid(model(xb.to(device))).cpu().numpy()
            probs.append(p)
    return np.concatenate(probs)

# Example scores on test split (first 10)
print("Sample probs:", score_events(X_use[te_idx])[:10])


Loaded: (293, 512, 2) labels: (293,)
No usable labels detected → generating SYNTHETIC planet anomalies
Data ready: (293, 512, 2), positives=102, negatives=191
Split → train=262 | test=31
pos_weight = 1.88
epoch 01 | train 0.9096 | test 0.9135
epoch 02 | train 0.9069 | test 0.9113
epoch 03 | train 0.9044 | test 0.9090
epoch 04 | train 0.9035 | test 0.9069
epoch 05 | train 0.9014 | test 0.9041
epoch 06 | train 0.8985 | test 0.8999
epoch 07 | train 0.8955 | test 0.8946
epoch 08 | train 0.8902 | test 0.8881
epoch 09 | train 0.8843 | test 0.8819
epoch 10 | train 0.8782 | test 0.8736
epoch 11 | train 0.8706 | test 0.8631
epoch 12 | train 0.8614 | test 0.8537
epoch 13 | train 0.8503 | test 0.8385
epoch 14 | train 0.8384 | test 0.8207
epoch 15 | train 0.8206 | test 0.8024
Test positives: 11 / 31
PR-AUC (Average Precision): 1.000
Confusion matrix:
 [[20  0]
 [ 2  9]]
              precision    recall  f1-score   support

           0      0.909     1.000     0.952        20
           1      1.

In [ ]:
import os, numpy as np, torch, torch.nn as nn

# ========= Conv1D-GRU (same arch you trained) =========
class Conv1DGRU(nn.Module):
    def __init__(self, in_ch=2, hid=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_ch, 32, kernel_size=7, padding=3), nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=5, padding=2),    nn.ReLU(),
            nn.MaxPool1d(2),  # 512 -> 256
            nn.Conv1d(64, 64, kernel_size=3, padding=1),    nn.ReLU(),
        )
        self.gru  = nn.GRU(input_size=64, hidden_size=64, batch_first=True, bidirectional=True)
        self.head = nn.Sequential(nn.Linear(128,64), nn.ReLU(), nn.Dropout(0.2), nn.Linear(64,1))
    def forward(self, x):        # x: (B, 2, 512)
        z = self.conv(x)         # (B, 64, 256)
        z = z.transpose(1, 2)    # (B, 256, 64)
        z, _ = self.gru(z)       # (B, 256, 128)
        z = z.mean(dim=1)        # (B, 128)
        return self.head(z).squeeze(1)

# -------- helpers (kept inside same cell for standalone use) --------
def _read_table(path):
    import pandas as pd
    try:
        df = pd.read_csv(path)
    except Exception:
        df = pd.read_csv(path, sep=r"\s+", engine="python")
    num = df.select_dtypes(include=[np.number])
    if num.shape[1] == 0:
        raise ValueError(f"No numeric columns found in {path}")
    return num.to_numpy()

def _to_2ch_512(arr, length=512, z_normalize=False):
    """Accepts shapes (T,2), (2,T), (T,1), (T,), (512,2) and returns (2,512) float32."""
    a = np.asarray(arr)
    if a.ndim == 1:                      # (T,)
        a = np.stack([a, np.zeros_like(a)], axis=1)        # -> (T,2)
    elif a.ndim == 2 and a.shape[0] == 2 and a.shape[1] != 2:
        a = a.T                                           # (2,T) -> (T,2)
    elif a.ndim == 2 and a.shape[1] == 1:                 # (T,1)
        a = np.concatenate([a, np.zeros_like(a)], axis=1) # -> (T,2)
    elif a.ndim != 2:
        raise ValueError(f"Unsupported array shape: {a.shape}")

    if a.shape[1] != 2:
        raise ValueError(f"Need 2 channels after prep, got shape {a.shape}")

    # resample along time to 512 if needed
    T = a.shape[0]
    if T != length:
        x_old = np.linspace(0, 1, T)
        x_new = np.linspace(0, 1, length)
        ch0 = np.interp(x_new, x_old, a[:,0])
        ch1 = np.interp(x_new, x_old, a[:,1])
        a = np.stack([ch0, ch1], axis=1)  # (512,2)

    if z_normalize:
        for c in range(2):
            mu, sd = a[:,c].mean(), a[:,c].std()
            a[:,c] = (a[:,c] - mu) / (sd + 1e-6)

    return a.T.astype("float32")  # -> (2,512)

def _load_model(model_path, device=None):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    ckpt = torch.load(model_path, map_location=device)
    cfg  = ckpt.get("config", {"in_ch":2, "hid":64})
    model = Conv1DGRU(in_ch=cfg.get("in_ch",2), hid=cfg.get("hid",64)).to(device)
    model.load_state_dict(ckpt["state_dict"])
    model.eval()
    return model, device

# ========= #1: single-event predictor =========
def predict_single_event(
    source,
    model_path="/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/models/conv1d_gru.pt",
    z_normalize=False,
):
    """
    source:  (a) numpy array, or
             (b) path to .npy / .csv / .txt / .tsv (will grab first 1–2 numeric cols)
             Accepts 1 or 2 channels; resamples to length 512.
    Returns: float prob in [0,1]
    """
    # load data
    if isinstance(source, str):
        ext = os.path.splitext(source)[1].lower()
        if ext == ".npy":
            arr = np.load(source)
        elif ext in [".csv", ".txt", ".tsv"]:
            arr = _read_table(source)
        else:
            raise ValueError(f"Unsupported file type: {ext}")
    else:
        arr = source

    x = _to_2ch_512(arr, length=512, z_normalize=z_normalize)   # (2,512)
    x = torch.from_numpy(x).unsqueeze(0)                         # (1,2,512)

    # load model + infer
    model, device = _load_model(model_path)
    with torch.no_grad():
        prob = torch.sigmoid(model(x.to(device))).item()
    return float(prob)

# ========= #2: multi-event predictor =========
def predict_many_events(
    sources,
    model_path="/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/models/conv1d_gru.pt",
    z_normalize=False,
    return_dataframe=True
):
    """
    sources: list of numpy arrays and/or file paths.
    Returns: list of probs (and a pandas DataFrame if return_dataframe=True)
    """
    xs = []
    names = []
    for src in sources:
        try:
            if isinstance(src, str):
                nm  = os.path.basename(src)
                ext = os.path.splitext(src)[1].lower()
                if ext == ".npy":
                    arr = np.load(src)
                elif ext in [".csv", ".txt", ".tsv"]:
                    arr = _read_table(src)
                else:
                    raise ValueError(f"Unsupported file type: {ext}")
            else:
                arr = src
                nm  = "array_" + str(len(xs))

            x = _to_2ch_512(arr, 512, z_normalize=z_normalize)   # (2,512)
            xs.append(x)
            names.append(nm)
        except Exception as e:
            # keep place with NaN if something fails
            xs.append(None); names.append(f"{nm if 'nm' in locals() else 'item'} (ERROR: {e})")

    # batch the valid ones
    valid_idx = [i for i,x in enumerate(xs) if x is not None]
    probs = [np.nan]*len(xs)
    if valid_idx:
        batch = torch.from_numpy(np.stack([xs[i] for i in valid_idx], axis=0))  # (B,2,512)
        model, device = _load_model(model_path)
        with torch.no_grad():
            p = torch.sigmoid(model(batch.to(device))).cpu().numpy().tolist()
        for k,i in enumerate(valid_idx):
            probs[i] = float(p[k])

    if return_dataframe:
        import pandas as pd
        df = pd.DataFrame({"source": names, "planet_prob": probs})
        return probs, df
    return probs


In [ ]:
# from .npy
p = predict_single_event("/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/demo_inputs/event_223_1ch.npy")
print("prob:", p)
p = predict_single_event("/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/demo_inputs/event_223_1ch.csv", z_normalize=True)
print("prob:", p)

# from a numpy array (T,2) or (512,2)
import numpy as np
arr = np.random.randn(512,2).astype("float32")
p = predict_single_event(arr)


prob: 0.4296358525753021
prob: 0.6621994376182556


In [ ]:
files = [
    "/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/demo_inputs/event_223_1ch.npy",
    "/content/drive/MyDrive/nasa_exoplanet/Microlensing/roman_2018/processed/demo_inputs/event_223_1ch.csv",
    np.random.randn(400,2)  # array with T!=512 → auto-resampled
]
probs, df = predict_many_events(files)
print(df.head())


              source  planet_prob
0  event_223_1ch.npy     0.429633
1  event_223_1ch.csv     0.429633
2            array_2     0.799596


In [ ]:
"""
Exoplanet Microlensing Classifier (Roman/WFIRST 2018)
TL;DR

Data: Roman (WFIRST) 2018 microlensing light curves (two filters: W149 & Z087), ~293 events per filter.

Prep: convert raw text to train-ready tensors (length-512, 2 channels), with robust cleaning + normalization.

Model: Conv1D → GRU → Dense binary classifier (planet anomaly vs normal), trained with a 90/10 split.

Labels: dataset has no planet labels → we inject synthetic “planet bumps” to show end-to-end training & inference.

Artifacts: X_2ch.npy, X_1ch.npy, y.npy, manifest.csv, meta.json, trained model conv1d_gru.pt, and ready-to-use inference functions.

1) Data acquisition
Code (download + extract)

Downloads 4 files into .../Microlensing/roman_2018/:

lc.tar.gz (all light curves, one text file per event per filter)

event_info.txt

wfirst_ephemeris_W149.txt

wfirst_ephemeris_Z087.txt

Uses a streaming downloader with progress, then a safe tar extractor (blocks path traversal).

After extract, does a quick count per filter.

Outputs you saw
✓ lc.tar.gz saved (101.8 MB)
✓ event_info.txt saved (0.0 MB)
✓ wfirst_ephemeris_W149.txt saved (2.8 MB)
✓ wfirst_ephemeris_Z087.txt saved (0.1 MB)
✓ Extracted lightcurves to: /.../roman_2018/lc
Found 293 W149 files and 293 Z087 files at .../roman_2018/lc

What that means

We’ve got 293 events in each filter → almost perfect 1:1 pairing.

Folder total after extraction: ~474.6 MB (your printout).
That’s normal: extraction + duplicated content across filters + text file overhead.

2) Data structure (raw)

Each light-curve text file has 3 columns (whitespace-delimited):

BJD   magnitude   error


BJD: barycentric Julian date (time)

mag: stellar brightness (smaller = brighter)

err: per-point uncertainty

Files are named like: ulwdc1_123_W149.txt and ulwdc1_123_Z087.txt.

3) Preprocessing → “train-ready” tensors
What we do (per event)

Pair filters by event id (e.g., ulwdc1_123).

Read both text files (if present).

Robust outlier clip (MAD) in magnitude space.

Compute median & MAD; drop points with |z| > 7 (tunable).

Convert mag → flux (more linear):

𝐹
∝
10
−
0.4
⋅
m
a
g
F∝10
−0.4⋅mag

Choose a time window:

If both filters exist: take the overlap region of their time spans (so channels line up).

Else: use the single filter’s full span.

Resample to a uniform grid of T = 512 points using linear interpolation.

Fix edge NaNs by forward/back fill, then zeros if anything remains.

Per-channel z-score standardization (zero mean, unit variance).

Emit tensors:

If both filters usable → shape (512, 2)

If only one usable → shape (512, 1)

Exports (what got saved)

processed/X_2ch.npy → all two-channel events, dtype float32, shape (N, 512, 2)

processed/X_1ch.npy → single-channel fallbacks, shape (M, 512, 1)

processed/y.npy → labels (-1 for “unknown” right now)

processed/manifest.csv → one row per attempted event with:

event_id, presence flags per filter, counts (n_W149, n_Z087), time spans, rough t0 per filter, and notes

processed/meta.json → how we processed the data (sequence length, channels, normalization, source link, etc.)

Outputs you saw
Discovered 293 events (keys), expected ~293.
... (processing) ...
2-channel samples: (293, 512, 2)
1-channel samples: (0, 512, 1)
Labels (y):       (293,)
Manifest rows:    293

Why this is solid

Everything is aligned and uniformly sampled → drop-in ready for any sequence model.

Two channels (W149 & Z087) kept separate → model can learn color dependence.

4) Model & training
Architecture: Conv1D → GRU → Dense

Conv1D stack (kernels 7/5/3 + MaxPool):

Learns local shapes (slopes, small bumps) & denoises a bit.

MaxPool halves length 512 → 256, keeps important patterns.

Bidirectional GRU (hidden=64 × 2 directions):

Captures temporal context (how bumps evolve).

Global mean over time → Dense → 1 logit (planet/not).

Why this arch? Convs are great for local features; GRU handles longer dependencies. For microlensing bumps (short anomalies within a longer curve), that combo slaps.

Labels situation & synthetic anomalies

The Roman 2018 pack doesn’t ship planet labels. So:

If real 0/1 labels exist in y.npy, we use them.

Else we inject synthetic “planet-like” Gaussian bumps into ~35% of curves to simulate positives:

random center t0, width 3–12, amplitude ~1–2.5σ.

slight channel mismatch to keep it realistic.

That lets us show the full ML loop (training, metrics, inference) in the hackathon even without ground truth.

Train/test split

Stratified 90/10 split on the synthetic labels.

BCEWithLogitsLoss + pos_weight for class balance (computed from the train split).

AdamW (lr 3e-4), EPOCHS=15.

Outputs you saw (training log)
Loaded: (293, 512, 2) labels: (293,)
No usable labels detected → generating SYNTHETIC planet anomalies
Data ready: (293, 512, 2), positives=102, negatives=191
Split → train=262 | test=31
pos_weight = 1.88
epoch 01 ... epoch 15 (test loss trending down)
Test positives: 11 / 31
PR-AUC (Average Precision): 1.000
Confusion matrix:
 [[20  0]
 [ 2  9]]
accuracy 0.935
Saved model -> .../processed/models/conv1d_gru.pt
Sample probs: [0.4278, 0.4284, 0.4278, 0.4928, 0.5438, ...]

How to interpret that

With synthetic labels, the model can learn bumps → hence the crisp PR-AUC.

Caveat: This is not an estimate of real-world performance; it just proves the pipeline + model are wired correctly and are sensitive to injected anomalies.

5) Inference (standalone, demo-safe)

You asked for two functions; here’s what they do (the code you pasted):

predict_single_event(source, model_path=..., z_normalize=False) → prob

Accepts:

numpy arrays or paths to .npy, .csv, .txt, .tsv

1-channel or 2-channel; short or long → it resamples to (2, 512)

Steps:

Reads first 1–2 numeric columns (for CSV/TXT).

Pads to 2 channels if you only give 1.

Resamples along time to 512.

Loads conv1d_gru.pt, runs forward pass, returns probability in [0,1].

predict_many_events(sources, ...) → (probs, dataframe)

Same as above, but for a list of arrays/paths.

Returns a list of probs and a small dataframe: source, planet_prob.

Any failed item becomes NaN with the error reason next to its name (handy in demos).

Why this rocks for the hackathon demo

You can point it at:

Your processed curves,

Any CSV with 1–2 numeric columns,

Any .npy array,
and it’ll “do the right thing” without extra massaging.

6) (Optional) Demo inputs helper

We also set up a utility to copy a few random events from X_2ch.npy into a demo_inputs/ folder as:

.npy (512×2)

.csv (two numeric columns: w149, z087)

plus two edge cases:

short sample (400 points) → shows auto-resample

1-channel sample → shows auto-channel-pad

So on stage you can do:

probs, df = predict_many_events(glob(".../processed/demo_inputs/*.csv"))
df.head()


and immediately get a table of planet probabilities.

7) What to say to judges (talk track)

Problem: detect planetary microlensing anomalies (tiny extra bump) in stellar light curves.

Data: Roman/WFIRST 2018 synthetic campaign — 293 paired events, two filters → perfect for multi-channel time series.

Pipeline: robust cleaning (MAD), mag→flux, uniform resampling (512), z-score per channel, produce tensors + manifest + metadata.

Model: Conv1D (local patterns) + GRU (temporal context) → single probability.

Labels: dataset ships unlabeled → injected synthetic planet bumps to validate end-to-end ML.

Result: model learns to flag bumps (great synthetic metrics).
Next step is swapping in real labels (OGLE/KMTNet planet events) or Roman-style challenge labels.

Demo: any CSV/NPY → predict_single_event(...) returns a probability live, backed by a saved PyTorch model.
"""